In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from sklearn.pipeline import Pipeline
import spacy
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix

import pandas as pd
from sklearn.model_selection import train_test_split
from proj_helper import Helper, TimeHelper
import joblib

from sklearn.model_selection import GridSearchCV

In [2]:
chat_data = pd.read_excel('UserChatData.xlsx', header=0) 

In [3]:
item = Helper()

for row in range(len(chat_data['Chat'])):
    chat_data['Chat'][row] = item.preprocess_givendata(chat_data['Chat'][row])


<ipython-input-3-7878766ba5c2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chat_data['Chat'][row] = item.preprocess_givendata(chat_data['Chat'][row])


In [4]:
shuffle_df = chat_data.sample(n=len(chat_data), random_state=21)
x_train,x_test,y_train,y_test=train_test_split(shuffle_df['Chat'],shuffle_df['Group'],test_size=0.20,random_state = 21)

In [5]:
pipe = Pipeline([('Count_Vect',CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('LR', LogisticRegression(n_jobs=1,max_iter=250))])
pipe.fit(x_train, y_train)


Pipeline(steps=[('Count_Vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('LR', LogisticRegression(max_iter=250, n_jobs=1))])

In [6]:
gs_params = [{'LR__C': [1e-1, 1, 1e3, 1e5],
             'LR__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
             'LR__penalty': ['l1', 'l2', 'elasticnet', 'none']}]
grid_search = GridSearchCV(estimator = pipe,
                               param_grid = gs_params,
                               scoring = 'accuracy',
                               cv = 4,
                               n_jobs = -1)
grid_search.fit(x_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

Best Accuracy: 92.17 %
Best Parameters: {'LR__C': 100000.0, 'LR__penalty': 'l1', 'LR__solver': 'saga'}


/Users/Chathuranga/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
